In [1]:
import requests
import pandas as pd
import json
import re
import numpy as np
import asyncio
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import time
from datetime import datetime
from IPython.display import clear_output


In [2]:
TICKERS_CSV = "data/top100/tickers.csv"  
OUTPUT_DIR = "output/top100"                         
MAX_TICKERS = 100

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:

df_tickers = pd.read_csv(TICKERS_CSV)

tickers = (
    df_tickers["symbol"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)#[:MAX_TICKERS]

tickers.sort()
tickers[:20]


['ACLX',
 'AGX',
 'ALV',
 'AMD',
 'AMKR',
 'ARWR',
 'ASH',
 'ASND',
 'ASTS',
 'AXON',
 'AXP',
 'BA',
 'BAC',
 'BBAR',
 'BBVA',
 'BELFB',
 'BKSY\t',
 'BLTE',
 'BMA',
 'BNS']

In [4]:
data = yf.download(
    tickers=" ".join(tickers),
    period="1d",
    interval="5m",
    group_by="ticker",
    threads=True,
    progress=False
)


C:\Users\a73s\AppData\Local\Temp\ipykernel_19496\1014172803.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: MEDIOLAN"}}}

2 Failed downloads:
['CS', 'MEDIOLAN']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')


In [5]:
records = []
timestamp = datetime.utcnow().isoformat()

for symbol in tickers:
    try:
        last_price = data[symbol]["Close"].dropna().iloc[-1]
        records.append({
            "symbol": symbol,
            "price": float(last_price),
            "timestamp": timestamp
        })
    except Exception:
        pass

prices_df = pd.DataFrame(records)
prices_df.head()


C:\Users\a73s\AppData\Local\Temp\ipykernel_19496\2810629483.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()


,symbol,price,timestamp
0,ACLX,67.239998,2026-02-11T10:30:20.634408
1,AGX,372.029999,2026-02-11T10:30:20.634408
2,ALV,125.820000,2026-02-11T10:30:20.634408
3,AMD,213.619995,2026-02-11T10:30:20.634408
4,AMKR,53.459999,2026-02-11T10:30:20.634408


In [6]:
csv_path = os.path.join(OUTPUT_DIR, "prices.csv")
prices_df.to_csv(csv_path, index=False)

print(f"Saved {csv_path}")

Saved output/top100\prices.csv


In [7]:
html = "<!DOCTYPE html>\n<html>\n<head>\n<meta charset='UTF-8'>\n<title>Market Dashboard</title>\n"
# Link to external CSS
html += "<link rel='stylesheet' href='https://hotstockmarket.web.app/styles.css'>\n"
html += "</head>\n<body>\n"

# Table
html += "<table class='table-container' border='1' cellpadding='5' cellspacing='0'>\n<thead><tr><th>Symbol</th><th>Price</th><th>Timestamp</th></tr></thead>\n<tbody>\n"
for _, row in prices_df.iterrows():
    html += f"<tr><td>{row['symbol']}</td><td>{row['price']:.2f}</td><td>{row['timestamp']}</td></tr>\n"
html += "</tbody>\n</table>\n"

# End HTML
html += "</body>\n</html>"

# Save HTML
HTML = os.path.join(OUTPUT_DIR, "dashboard.html")
with open(HTML, "w", encoding="utf-8") as f:
    f.write(html)

print("HTML file generated: dashboard.html")


HTML file generated: dashboard.html


In [8]:
from datetime import datetime
import matplotlib.pyplot as plt
import os

for symbol in tickers:
    try:
        series = data[symbol]["Close"].dropna()

        if series.empty:
            continue

        last_price = series.iloc[-1]
        last_time = series.index[-1]

        plt.figure(figsize=(6, 4))
        plt.plot(series.index, series.values, linewidth=1)
        plt.grid(True)

        # Title
        plt.title(f"{symbol}", fontsize=12)

        # Last price marker
        plt.scatter(last_time, last_price, zorder=3)

        # Last price label
        plt.text(
            last_time,
            last_price,
            f" {last_price:.2f}",
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="left",
            bbox=dict(facecolor="white", edgecolor="none", alpha=0.8)
        )

        plt.tight_layout()

        chart_path = os.path.join(OUTPUT_DIR, f"{symbol}.png")
        plt.savefig(chart_path, dpi=150)
        plt.close()

        print(f"Saved {chart_path}")

    except Exception as e:
        print(f"Skipping {symbol}: {e}")


Saved output/top100\ACLX.png
Saved output/top100\AGX.png
Saved output/top100\ALV.png
Saved output/top100\AMD.png
Saved output/top100\AMKR.png
Saved output/top100\ARWR.png
Saved output/top100\ASH.png
Saved output/top100\ASND.png
Saved output/top100\ASTS.png
Saved output/top100\AXON.png
Saved output/top100\AXP.png
Saved output/top100\BA.png
Saved output/top100\BAC.png
Saved output/top100\BBAR.png
Saved output/top100\BBVA.png
Saved output/top100\BELFB.png
Skipping BKSY	: 'BKSY\t'
Saved output/top100\BLTE.png
Saved output/top100\BMA.png
Saved output/top100\BNS.png
Saved output/top100\C.png
Saved output/top100\CAE.png
Saved output/top100\CIEN.png
Saved output/top100\CM.png
Saved output/top100\DB.png
Saved output/top100\DIOD.png
Saved output/top100\DSGX.png
Saved output/top100\ENVA.png
Saved output/top100\EOSE.png
Saved output/top100\FROG.png
Saved output/top100\GILT.png
Saved output/top100\GS.png
Saved output/top100\HSBC.png
Saved output/top100\IBKR.png
Saved output/top100\ING.png
Skipping 